# Bar data

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('20.230.202.72', 7497, clientId=50022)

<IB connected to 20.230.202.72:7497 clientId=50022>

## Historical data

To get the earliest date of available bar data the "head timestamp" can be requested:

In [2]:
# contract = Stock('TSLA', 'SMART', 'USD')
# contract = Future(symbol="NQ", localSymbol="NQU3", exchange="CME")
# contract = Future(symbol="CL", localSymbol="CLQ3", exchange="NYMEX")
# contract = Future(symbol="VXM", localSymbol="VXMN3", exchange="CFE")
contract = Forex('EURUSD')
contract = ib.qualifyContracts(contract)
print(contract)
# ib.reqHeadTimeStamp(contract, whatToShow='TRADES', useRTH=True)

[Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')]


In [4]:
op = Option(
            symbol="QQQ",
            exchange="SMART",
            currency="USD",
            right="C",
            lastTradeDateOrContractMonth="20230724",
            strike="384",
            multiplier=100,
        )

[op] = ib.qualifyContracts(op)
print(op)

Option(conId=641396743, symbol='QQQ', lastTradeDateOrContractMonth='20230724', strike=384.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='QQQ   230724C00384000', tradingClass='QQQ')


In [10]:
ticker = ib.reqTickers(op)
print(ticker)

[Ticker(contract=Option(conId=641396743, symbol='QQQ', lastTradeDateOrContractMonth='20230724', strike=384.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='QQQ   230724C00384000', tradingClass='QQQ'), time=datetime.datetime(2023, 7, 19, 19, 42, 38, 4995, tzinfo=datetime.timezone.utc), minTick=0.01, bid=3.99, bidSize=334.0, bidExchange='INQZTHEJP', ask=4.03, askSize=1192.0, askExchange='ACIXNBQZWTHEJP', last=4.07, lastSize=1.0, prevBid=4.0, prevBidSize=262.0, prevAsk=4.04, prevAskSize=1205.0, volume=724.0, high=5.48, low=3.27, close=4.32, halted=0.0, bidGreeks=OptionComputation(tickAttrib=0, impliedVol=0.15785774947368883, delta=0.6166928332529255, optPrice=3.990000009536743, pvDividend=0.0, gamma=0.05364761516009434, vega=0.17085331809572502, theta=-0.3065255316029573, undPrice=385.77), askGreeks=OptionComputation(tickAttrib=0, impliedVol=0.1602009445242762, delta=0.6151579378458818, optPrice=4.03000020980835, pvDividend=0.0, gamma=0.05293603747295739, veg

Peer closed connection.


In [9]:
ticker = ib.reqMktData(op)
print(ticker)

Ticker(contract=Option(conId=641396743, symbol='QQQ', lastTradeDateOrContractMonth='20230724', strike=384.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='QQQ   230724C00384000', tradingClass='QQQ'), time=datetime.datetime(2023, 7, 19, 19, 42, 32, 720185, tzinfo=datetime.timezone.utc), minTick=0.01, bid=3.99, bidSize=576.0, bidExchange='INBQZTHEJP', ask=4.03, askSize=906.0, askExchange='AIXNQZWTHEJP', last=4.07, lastSize=1.0, prevBid=4.0, prevBidSize=920.0, prevAsk=4.04, prevAskSize=666.0, volume=724.0, high=5.48, low=3.27, close=4.32, ticks=[TickData(time=datetime.datetime(2023, 7, 19, 19, 42, 32, 720185, tzinfo=datetime.timezone.utc), tickType=0, price=3.99, size=576.0), TickData(time=datetime.datetime(2023, 7, 19, 19, 42, 32, 720185, tzinfo=datetime.timezone.utc), tickType=3, price=4.03, size=906.0)], bidGreeks=OptionComputation(tickAttrib=0, impliedVol=0.15785774947368883, delta=0.6166928332529255, optPrice=3.990000009536743, pvDividend=0.0, gamma=0.05

To request hourly data of the last 60 trading days:

In [ ]:
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting='5 secs',
        whatToShow='TRADES',
        useRTH=False,
        formatDate=1)

In [ ]:
bars[0]

In [ ]:
bars[-1]

Convert the list of bars to a data frame and print the first and last rows:

In [ ]:
len(bars)

In [ ]:
df = util.df(bars)

display(df.head())
display(df.tail())

Instruct the notebook to draw plot graphics inline:

In [ ]:
%matplotlib inline

Plot the close data

In [ ]:
df.plot(y='close');

There is also a utility function to plot bars as a candlestick plot. It can accept either a DataFrame or a list of bars. Here it will print the last 100 bars:

In [ ]:
util.barplot(bars[-100:], title=contract.symbol);

## Historical data with realtime updates

A new feature of the API is to get live updates for historical bars. This is done by setting `endDateTime` to an empty string and the `keepUpToDate` parameter to `True`.

Let's get some bars with an keepUpToDate subscription:

In [ ]:
contract = Forex('EURUSD')

bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='900 S',
        barSizeSetting='10 secs',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=1,
        keepUpToDate=True)

Replot for every change of the last bar:

In [ ]:
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

def onBarUpdate(bars, hasNewBar):
    plt.close()
    plot = util.barplot(bars)
    clear_output(wait=True)
    display(plot)

bars.updateEvent += onBarUpdate

ib.sleep(10)
ib.cancelHistoricalData(bars)

Realtime bars
------------------

With ``reqRealTimeBars`` a subscription is started that sends a new bar every 5 seconds.

First we'll set up a event handler for bar updates:

In [ ]:
def onBarUpdate(bars, hasNewBar):
    print(bars[-1])

Then do the real request and connect the event handler,

In [ ]:
contract = Forex("EURUSD")

In [ ]:
bars = ib.reqRealTimeBars(contract, 5, 'MIDPOINT', False)
bars.updateEvent += onBarUpdate

let it run for half a minute and then cancel the realtime bars.

In [ ]:
ib.sleep(30)
ib.cancelRealTimeBars(bars)

The advantage of reqRealTimeBars is that it behaves more robust when the connection to the IB server farms is interrupted. After the connection is restored, the bars from during the network outage will be backfilled and the live bars will resume.

reqHistoricalData + keepUpToDate will, at the moment of writing, leave the whole API inoperable after a network interruption.

In [ ]:
ib.disconnect()